In [7]:
pip install konlpy 

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
import re
from konlpy.tag import Mecab
from konlpy.tag import Okt
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences

In [29]:
def load_data(path):
    """
    path = 'D:/AIB/CP1/여행계획.xlsx'
    """
    df = pd.DataFrame()
    data = pd.read_excel(path)
    return data

In [39]:
data = load_data('D:/AIB/CP1/벚꽃놀이.xlsx')

In [40]:
data.head()

,list_name,name,views,comment,like_num
0,[장기연애],벚꽃놀이,"조회수 5,391,893회",김치찌개 드실때부터 생각한건데 음식을 너무 찰지게 드심,1.8만
1,[장기연애],벚꽃놀이,"조회수 5,391,893회",떨어진 꽃잎을 모으는 것만큼 아름다운 것은 없어 I love it\n,1.4천
2,[장기연애],벚꽃놀이,"조회수 5,391,893회",1:13 자기 자리만 정성스럽게 먼지털고1:43 휴지 대신 낙옆 주워서 닦으라고 주...,8.2천
3,[장기연애],벚꽃놀이,"조회수 5,391,893회",봉지 돌리면서 뛰어오시는거 너무 웃기고 기여우심 ㅋㅋㅋ,4.3천
4,[장기연애],벚꽃놀이,"조회수 5,391,893회",벚꽃 뿌려달라 할때 나무 발로 차는게 아니라 떨어진 꽃잎 모아서 쓴거 착해ㅜㅋㅋㅋ,4.6천


In [59]:
def emoticon_del(data):
    #이모티콘 제거
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    #분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
    han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~,".\n\r#\ufeff\u200d]')

    comment_list = []
    for i in range(len(data)):
        comment_list.append(data['comment'].iloc[i])

    comment_result = []
    for i in comment_list:
        tokens = re.sub(emoji_pattern,"",str(i))
        tokens = re.sub(han,"",tokens)
        comment_result.append(tokens)
    comment_result = pd.DataFrame(comment_result, columns=["comment"])
    data['comment'] = comment_result 

    return data


In [61]:
data

,list_name,name,views,comment,like_num
0,[장기연애],벚꽃놀이,"조회수 5,391,893회",김치찌개 드실때부터 생각한건데 음식을 너무 찰지게 드심,1.8만
1,[장기연애],벚꽃놀이,"조회수 5,391,893회",떨어진 꽃잎을 모으는 것만큼 아름다운 것은 없어 I love it,1.4천
2,[장기연애],벚꽃놀이,"조회수 5,391,893회",1:13 자기 자리만 정성스럽게 먼지털고1:43 휴지 대신 낙옆 주워서 닦으라고 주...,8.2천
3,[장기연애],벚꽃놀이,"조회수 5,391,893회",봉지 돌리면서 뛰어오시는거 너무 웃기고 기여우심,4.3천
4,[장기연애],벚꽃놀이,"조회수 5,391,893회",벚꽃 뿌려달라 할때 나무 발로 차는게 아니라 떨어진 꽃잎 모아서 쓴거 착해,4.6천
...,...,...,...,...,...
2195,[장기연애],벚꽃놀이,"조회수 5,391,893회",힝야,0
2196,[장기연애],벚꽃놀이,"조회수 5,391,893회",벤치 닦고 지가 앉아,0
2197,[장기연애],벚꽃놀이,"조회수 5,391,893회",용현동 신도브래뉴 오르막길 전체가 벚꽃입니다 오세요,0
2198,[장기연애],벚꽃놀이,"조회수 5,391,893회",한시간만에 30만,0


In [60]:
data = emoticon_del(data)

In [62]:
def tokenizer(data):
    # 불용어(포함 안 시킬 단어) 처리 
    stopwords = ['도', '는', '다', '의', '가', '이', '은', '한', '에', '하', '고', '을', '를', '인', '듯', '과', '와', '네', '들', '듯', '지', 
    '임', '게', '만', '게임', '겜', '되', '음', '면', '!', '.', ',', '~', '?', 'ㅡ', 'ㅠ', 'ㅜ']

    ##C드라이브 Mecab이 설치되어 있는 곳 -> dicpath로 넣어주어야함
    mecab = Mecab(dicpath=r"C:/mecab/mecab-ko-dic") 
    
    data['tokenized'] = data['comment'].apply(mecab.morphs)
    data['tokenized'] = data['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

In [63]:
tokenizer(data)

In [64]:
data

,list_name,name,views,comment,like_num,tokenized
0,[장기연애],벚꽃놀이,"조회수 5,391,893회",김치찌개 드실때부터 생각한건데 음식을 너무 찰지게 드심,1.8만,"[김치찌개, 드실, 때, 부터, 생각, 건데, 음식, 너무, 찰지, 드, 심]"
1,[장기연애],벚꽃놀이,"조회수 5,391,893회",떨어진 꽃잎을 모으는 것만큼 아름다운 것은 없어 I love it,1.4천,"[떨어진, 꽃잎, 모으, 것, 만큼, 아름다운, 것, 없, 어, I, love, it]"
2,[장기연애],벚꽃놀이,"조회수 5,391,893회",1:13 자기 자리만 정성스럽게 먼지털고1:43 휴지 대신 낙옆 주워서 닦으라고 주...,8.2천,"[1, :, 13, 자기, 자리, 정성, 스럽, 먼지, 털, 1, :, 43, 휴지..."
3,[장기연애],벚꽃놀이,"조회수 5,391,893회",봉지 돌리면서 뛰어오시는거 너무 웃기고 기여우심,4.3천,"[봉지, 돌리, 면서, 뛰어오, 시, 거, 너무, 웃기, 기여, 우심]"
4,[장기연애],벚꽃놀이,"조회수 5,391,893회",벚꽃 뿌려달라 할때 나무 발로 차는게 아니라 떨어진 꽃잎 모아서 쓴거 착해,4.6천,"[벚꽃, 뿌려, 달, 라, 할, 때, 나무, 발, 로, 차, 아니, 라, 떨어진, ..."
...,...,...,...,...,...,...
2195,[장기연애],벚꽃놀이,"조회수 5,391,893회",힝야,0,"[힝, 야]"
2196,[장기연애],벚꽃놀이,"조회수 5,391,893회",벤치 닦고 지가 앉아,0,"[벤치, 닦, 앉, 아]"
2197,[장기연애],벚꽃놀이,"조회수 5,391,893회",용현동 신도브래뉴 오르막길 전체가 벚꽃입니다 오세요,0,"[용현동, 신, 브, 래, 뉴, 오르막길, 전체, 벚꽃, 입니다, 오, 세, 요]"
2198,[장기연애],벚꽃놀이,"조회수 5,391,893회",한시간만에 30만,0,"[시간, 30]"


In [32]:
def to_number(data):
    # views 수정
    data['views'] = data['views'].apply(lambda x: re.sub('[^0-9]', "", x)).astype(int)

    # like_num 수정
    def like_arrage(x):
        if type(x) == float or type(x) == int:
            return x 
        if '만' in x:
            if len(x) > 1:
                return float(x.replace('만','')) * 10000
            return x
        if '천' in x:
            if len(x) > 1:
                return float(x.replace('천','')) * 1000
        return x
    
    data['like_num'] = data['like_num'].apply(like_arrage).astype(int)  

In [65]:
to_number(data)

In [66]:
data

,list_name,name,views,comment,like_num,tokenized
0,[장기연애],벚꽃놀이,5391893,김치찌개 드실때부터 생각한건데 음식을 너무 찰지게 드심,18000,"[김치찌개, 드실, 때, 부터, 생각, 건데, 음식, 너무, 찰지, 드, 심]"
1,[장기연애],벚꽃놀이,5391893,떨어진 꽃잎을 모으는 것만큼 아름다운 것은 없어 I love it,1400,"[떨어진, 꽃잎, 모으, 것, 만큼, 아름다운, 것, 없, 어, I, love, it]"
2,[장기연애],벚꽃놀이,5391893,1:13 자기 자리만 정성스럽게 먼지털고1:43 휴지 대신 낙옆 주워서 닦으라고 주...,8200,"[1, :, 13, 자기, 자리, 정성, 스럽, 먼지, 털, 1, :, 43, 휴지..."
3,[장기연애],벚꽃놀이,5391893,봉지 돌리면서 뛰어오시는거 너무 웃기고 기여우심,4300,"[봉지, 돌리, 면서, 뛰어오, 시, 거, 너무, 웃기, 기여, 우심]"
4,[장기연애],벚꽃놀이,5391893,벚꽃 뿌려달라 할때 나무 발로 차는게 아니라 떨어진 꽃잎 모아서 쓴거 착해,4600,"[벚꽃, 뿌려, 달, 라, 할, 때, 나무, 발, 로, 차, 아니, 라, 떨어진, ..."
...,...,...,...,...,...,...
2195,[장기연애],벚꽃놀이,5391893,힝야,0,"[힝, 야]"
2196,[장기연애],벚꽃놀이,5391893,벤치 닦고 지가 앉아,0,"[벤치, 닦, 앉, 아]"
2197,[장기연애],벚꽃놀이,5391893,용현동 신도브래뉴 오르막길 전체가 벚꽃입니다 오세요,0,"[용현동, 신, 브, 래, 뉴, 오르막길, 전체, 벚꽃, 입니다, 오, 세, 요]"
2198,[장기연애],벚꽃놀이,5391893,한시간만에 30만,0,"[시간, 30]"


In [33]:
def output(data, file_name):
    data.to_excel(f"./{file_name}.xlsx", index = False)

In [67]:
output(data, '벚꽃놀이(tok)')

$여기까지만 확인! 밑에는 GARBAGE$


In [2]:
#데이터 불러오기
df = pd.DataFrame()

basicpath = 'D:/AIB/CP1/여행계획.xlsx'
data = pd.read_excel(basicpath)

data.head()

,list_name,name,views,youtube_id,comment,like_num
0,[장기연애],여행계획,"조회수 9,425,237회",엄지렐라 Umjirella,유튜브 하는 거 진짜 비밀로 해라,4.9만
1,[장기연애],여행계획,"조회수 9,425,237회",치트키TV,동생한텐 다리 처먹었다고 욕하지만남친한텐 다리 헤비하다고 양보하는거봐...스윗,4.1만
2,[장기연애],여행계획,"조회수 9,425,237회",Aesthetic life,장기연애 한 적은 한 번도 없지만 이 영상 볼 때마다 힐링된다.. 둘이 치킨 맛있게...,3.5천
3,[장기연애],여행계획,"조회수 9,425,237회",SR Kim,영혼 없는 리액션이라도 해주는 거 넘 좋다. 의외의 힐링 재질.,703
4,[장기연애],여행계획,"조회수 9,425,237회",달토키,진짜 찐 오래된 장기연애 커플 보는 것 같음ㅎㅎ서로의 티키타카가 너무 찰떡같고서로 ...,1.7천


In [3]:
test = data

In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3279 entries, 0 to 3278
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   list_name   3279 non-null   object
 1   name        3279 non-null   object
 2   views       3279 non-null   object
 3   youtube_id  3279 non-null   object
 4   comment     3279 non-null   object
 5   like_num    3279 non-null   object
dtypes: object(6)
memory usage: 153.8+ KB


In [45]:
#이모티콘 제거
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

#분석에 어긋나는 불용어구 제외 (특수문자, 의성어)
han = re.compile(r'[ㄱ-ㅎㅏ-ㅣ!?~,".\n\r#\ufeff\u200d]')

In [52]:
comment_list = []
for i in range(len(test)):
    comment_list.append(test['comment'].iloc[i])

In [51]:
comment_result = []

for i in comment_list:
    tokens = re.sub(emoji_pattern,"",i)
    tokens = re.sub(han,"",tokens)
    comment_result.append(tokens)


comment_result = pd.DataFrame(comment_result, columns=["comment"])

TypeError: expected string or bytes-like object

In [8]:
test['comment'] = comment_result 

In [9]:
comment_result
test

,list_name,name,views,youtube_id,comment,like_num
0,[장기연애],여행계획,"조회수 9,425,237회",엄지렐라 Umjirella,유튜브 하는 거 진짜 비밀로 해라,4.9만
1,[장기연애],여행계획,"조회수 9,425,237회",치트키TV,동생한텐 다리 처먹었다고 욕하지만남친한텐 다리 헤비하다고 양보하는거봐스윗,4.1만
2,[장기연애],여행계획,"조회수 9,425,237회",Aesthetic life,장기연애 한 적은 한 번도 없지만 이 영상 볼 때마다 힐링된다 둘이 치킨 맛있게 먹...,3.5천
3,[장기연애],여행계획,"조회수 9,425,237회",SR Kim,영혼 없는 리액션이라도 해주는 거 넘 좋다 의외의 힐링 재질,703
4,[장기연애],여행계획,"조회수 9,425,237회",달토키,진짜 찐 오래된 장기연애 커플 보는 것 같음서로의 티키타카가 너무 찰떡같고서로 길게...,1.7천
...,...,...,...,...,...,...
3274,[장기연애],여행계획,"조회수 9,425,237회",Hazel J,경험에서 나온 연기,0
3275,[장기연애],여행계획,"조회수 9,425,237회",노래하는요원,비밀은 지켜야지요,0
3276,[장기연애],여행계획,"조회수 9,425,237회",jh,기믹이 뭐에요,0
3277,[장기연애],여행계획,"조회수 9,425,237회",ᄒᄒ어피치,여기치킨어디꺼예오,0


In [5]:
data['comment'] = data['comment'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
data['comment'].replace('', np.nan, inplace=True)
print(data.isnull().sum())

list_name     0
name          0
views         0
youtube_id    0
comment       3
like_num      0
dtype: int64


C:\Users\Huiseop\AppData\Local\Temp\ipykernel_24956\2762186075.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data['comment'] = data['comment'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3279 entries, 0 to 3278
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   list_name   3279 non-null   object
 1   name        3279 non-null   object
 2   views       3279 non-null   object
 3   youtube_id  3279 non-null   object
 4   comment     3276 non-null   object
 5   like_num    3279 non-null   object
dtypes: object(6)
memory usage: 153.8+ KB


In [14]:
from pykospacing import spacing
from konlpy.tag import kkma

ImportError: cannot import name 'spacing' from 'pykospacing' (c:\Users\Huiseop\anaconda3\envs\cp1\lib\site-packages\pykospacing\__init__.py)

In [12]:
def preprocessing(review):
    total_review = ''
    #인풋리뷰
    for idx in range(len(review)):
        r = review[idx]
        #하나의 리뷰에서 문장 단위로 자르기
        for sentence in kkma.sentences(r):
            sentence = re.sub('([a-zA-Z])','',sentence)
            sentence = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',sentence)
            sentence = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',sentence)
            if len(sentence) == 0:
                continue
            if len(sentence) < 198:
                sentence = spacing(sentence)
            sentence += '. '
            total_review += sentence
    return total_review

In [13]:
data['comment'] = data['comment'].apply(preprocessing)

NameError: name 'kkma' is not defined

In [10]:
# 불용어(포함 안 시킬 단어) 처리 
stopwords = ['도', '는', '다', '의', '가', 
'이', '은', '한', '에', '하', '고', '을', 
'를', '인', '듯', '과', '와', '네', '들', 
'듯', '지', '임', '게', '만', '게임', 
'겜', '되', '음', '면', '!', '.', ',', '~', '?'
'ㅡ', 'ㅠ', 'ㅜ']

In [12]:
#C드라이브 Mecab이 설치되어 있는 곳 -> dicpath로 넣어주어야함
mecab = Mecab(dicpath=r"C:/mecab/mecab-ko-dic") 

test['tokenized'] = test['comment'].apply(mecab.morphs)
test['tokenized'] = test['tokenized'].apply(lambda x: [item for item in x if item not in stopwords])

In [14]:
test

,list_name,name,views,youtube_id,comment,like_num,tokenized
0,[장기연애],여행계획,"조회수 9,425,237회",엄지렐라 Umjirella,유튜브 하는 거 진짜 비밀로 해라,4.9만,"[유튜브, 거, 진짜, 비밀, 로, 해라]"
1,[장기연애],여행계획,"조회수 9,425,237회",치트키TV,동생한텐 다리 처먹었다고 욕하지만남친한텐 다리 헤비하다고 양보하는거봐스윗,4.1만,"[동생, 한텐, 다리, 처먹, 었, 다고, 욕, 지만, 남친, 한텐, 다리, 헤비,..."
2,[장기연애],여행계획,"조회수 9,425,237회",Aesthetic life,장기연애 한 적은 한 번도 없지만 이 영상 볼 때마다 힐링된다 둘이 치킨 맛있게 먹...,3.5천,"[장기연, 애, 적, 번, 없, 지만, 영상, 볼, 때, 마다, 힐링, 된다, 둘,..."
3,[장기연애],여행계획,"조회수 9,425,237회",SR Kim,영혼 없는 리액션이라도 해주는 거 넘 좋다 의외의 힐링 재질,703,"[영혼, 없, 리액션, 라도, 해, 주, 거, 넘, 좋, 의외, 힐링, 재질]"
4,[장기연애],여행계획,"조회수 9,425,237회",달토키,진짜 찐 오래된 장기연애 커플 보는 것 같음서로의 티키타카가 너무 찰떡같고서로 길게...,1.7천,"[진짜, 찐, 오래, 된, 장기, 연애, 커플, 보, 것, 같, 서로, 티키, 타카..."
...,...,...,...,...,...,...,...
3274,[장기연애],여행계획,"조회수 9,425,237회",Hazel J,경험에서 나온 연기,0,"[경험, 에서, 나온, 연기]"
3275,[장기연애],여행계획,"조회수 9,425,237회",노래하는요원,비밀은 지켜야지요,0,"[비밀, 지켜야, 요]"
3276,[장기연애],여행계획,"조회수 9,425,237회",jh,기믹이 뭐에요,0,"[기, 믹, 뭐, 요]"
3277,[장기연애],여행계획,"조회수 9,425,237회",ᄒᄒ어피치,여기치킨어디꺼예오,0,"[여기, 치킨, 어디, 꺼, 예오]"


In [15]:
# views 수정 format = 조회수 9,425,237회
import re
# 정규표현식으로 숫자만 뽑아내기? 
views_regex = re.compile('[^0-9]')

test['views'] = test['views'].apply(lambda x: re.sub('[^0-9]', "", x)).astype(int)

In [16]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3279 entries, 0 to 3278
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   list_name   3279 non-null   object
 1   name        3279 non-null   object
 2   views       3279 non-null   int32 
 3   youtube_id  3279 non-null   object
 4   comment     3279 non-null   object
 5   like_num    3279 non-null   object
 6   tokenized   3279 non-null   object
dtypes: int32(1), object(6)
memory usage: 166.6+ KB


In [17]:
# like_num 수정 format N.M만, N.M천 
def like_arrage(x):
    if type(x) == float or type(x) == int:
      return x 
    if '만' in x:
      if len(x) > 1:
        return float(x.replace('만','')) * 10000
      return x
    if '천' in x:
      if len(x) > 1:
        return float(x.replace('천','')) * 1000
    return x

In [18]:
data.head()
test['like_num'] = test['like_num'].apply(like_arrage).astype(int)

In [19]:
test.head()

,list_name,name,views,youtube_id,comment,like_num,tokenized
0,[장기연애],여행계획,9425237,엄지렐라 Umjirella,유튜브 하는 거 진짜 비밀로 해라,49000,"[유튜브, 거, 진짜, 비밀, 로, 해라]"
1,[장기연애],여행계획,9425237,치트키TV,동생한텐 다리 처먹었다고 욕하지만남친한텐 다리 헤비하다고 양보하는거봐스윗,41000,"[동생, 한텐, 다리, 처먹, 었, 다고, 욕, 지만, 남친, 한텐, 다리, 헤비,..."
2,[장기연애],여행계획,9425237,Aesthetic life,장기연애 한 적은 한 번도 없지만 이 영상 볼 때마다 힐링된다 둘이 치킨 맛있게 먹...,3500,"[장기연, 애, 적, 번, 없, 지만, 영상, 볼, 때, 마다, 힐링, 된다, 둘,..."
3,[장기연애],여행계획,9425237,SR Kim,영혼 없는 리액션이라도 해주는 거 넘 좋다 의외의 힐링 재질,703,"[영혼, 없, 리액션, 라도, 해, 주, 거, 넘, 좋, 의외, 힐링, 재질]"
4,[장기연애],여행계획,9425237,달토키,진짜 찐 오래된 장기연애 커플 보는 것 같음서로의 티키타카가 너무 찰떡같고서로 길게...,1700,"[진짜, 찐, 오래, 된, 장기, 연애, 커플, 보, 것, 같, 서로, 티키, 타카..."
